# Homework 4

### Подготовка

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [10]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [12]:
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [13]:
recommender = MainRecommender(data_train, item_features)

In [14]:
%%time
result['als_bm25'] = result['user_id'].apply(lambda x: recommender.get_recommendations(x, N=5))

CPU times: user 1.72 s, sys: 115 ms, total: 1.83 s
Wall time: 928 ms


In [15]:
%%time
result['similar_items'] = result['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=5))

CPU times: user 5.76 s, sys: 1.31 s, total: 7.08 s
Wall time: 3.55 s


In [16]:
%%time
result['similar_users'] = result['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=5))

CPU times: user 26.5 s, sys: 6.58 s, total: 33.1 s
Wall time: 17.1 s


In [17]:
result.head()

,user_id,actual,als_bm25,similar_items,similar_users
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[923789, 888178, 1131488, 966684, 944836]","[5568447, 966348, 1075214, 1017127, 842769]","[1023196, 1017127, 1010259, 862535, 982755]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[999999, 949142, 5570383, 969568, 865992]","[1003487, 968072, 1025535, 902172, 969568]","[12172240, 965530, 1046587, 5587656, 964133]"
2,5,"[913077, 1118028, 1386668]","[1130581, 935393, 1072968, 863885, 995151]","[1105301, 920109, 1047769, 1058404, 1139786]","[1037965, 1107973, 823704, 1128016, 972445]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[1006878, 999999, 1034176, 9420181, 10149640]","[848029, 9677366, 878715, 8020234, 8019273]","[849202, 882247, 888586, 972416, 9802981]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[1128016, 947326, 5591170, 1018769, 1018007]","[1050534, 1094223, 1065979, 1131625, 947326]","[907631, 944836, 862799, 1031316, 1094223]"


In [27]:
for temp_name in result.columns[2:]:
    print(f'{temp_name}, precision@5: {result.apply(lambda row: precision_at_k(row[temp_name], row["actual"]), axis=1).mean()}')

als_bm25, precision@5: 0.02651933701657467
similar_items, precision@5: 0.016172777498744313
similar_users, precision@5: 0.014264188849824177


Пока результат оставляет желать лучшего, однако считать метрики по рекомендациям, которые пользователь еще никогда не получал не совсем корректно.
Нужно будет поиграться с фильтрацией.

----